# Import package

In [1]:
import json
from pathlib import Path

import bentoml
import torch
from loguru import logger
from transformers import AutoTokenizer

from attribute_extraction.models.attribute_classification import MultiAttributeClassifier
from attribute_extraction.models.mapper import Mapper

# Setting up variables

In [31]:
projection_dim = 256

In [33]:
path = "/home/20014946/Documents/Final_Project_PEM/Final_Project_PEM/outputs_train_workflow"
hyper_parameters = json.load(open(f"{path}/hyper_parameters.json"))



In [39]:
tokenizer = AutoTokenizer.from_pretrained(hyper_parameters["model_name"])

saved_tokenizer = bentoml.picklable_model.save_model(
    "attribute-classification-tokenizer",
    tokenizer,
    signatures={"__call__": {"batchable": False}},
)

logger.info(f"tokenizer saved: {saved_tokenizer}")



2023-05-23 14:39:58.444 | INFO     | __main__:<module>:9 - tokenizer saved: Model(tag="attribute-classification-tokenizer:5u7su7hzm2cabv5j")


In [41]:
mapper = Mapper.load(f"{path}/mapper.json")

model = MultiAttributeClassifier(
    vocab_size=tokenizer.vocab_size,
    class_config=mapper.mappings,
    projection_dim=projection_dim,
)

model.load_state_dict(
    torch.load(
        path / "model.ckpt",
        map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    )["state_dict"]
)

model.eval()

saved_model = bentoml.picklable_model.save_model(
    "attribute-classification-model",
    model,
    signatures={"predict_masked": {"batchable": False}},
)

logger.info(f"Model saved: {saved_model}")

TypeError: __init__() missing 1 required positional argument: 'model_name_or_path'